In [26]:

import numpy as np
import pandas as pd 
import re
import nltk
import string,time
import ast
from sklearn.metrics.pairwise import cosine_similarity
from textblob import TextBlob
from nltk.tokenize import word_tokenize,sent_tokenize


In [27]:

df = pd.read_csv('./citation_network.csv')
# Lấy 10.000 dòng đầu tiên
#df = df.head(10000)  # Sử dụng phương thức head

C:\Users\HP\AppData\Local\Temp\ipykernel_15588\4208446275.py:1: DtypeWarning: Columns (0,3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./citation_network.csv')


In [28]:
#shape of the dataframe

df.shape

(4894402, 8)

In [29]:
df.head()

,id,title,year,n_citation,doc_type,reference_count,references,doi
0,1091,Preliminary Design of a Network Protocol Learn...,2013.0,1,Conference,2.0,2005687710;2018037215,https://doi.org/10.1007/978-3-642-39476-8_19
1,1388,Further Results on Independence in Direct-Prod...,2000.0,1,Journal,NaN,NaN,NaN
2,1674,A methodology for the physically accurate visu...,2011.0,1,Conference,15.0,1535888970;1992876689;1993710814;2035653341;20...,https://doi.org/10.2312/VAST/VAST11/137-144
3,1688,"Comparison of GARCH, Neural Network and Suppor...",2009.0,6,Conference,3.0,1560724230;1986968751;2156909104,https://doi.org/10.1007/978-3-642-11164-8_97
4,5411,COMPARING GNG3D AND QUADRIC ERROR METRICS METH...,2009.0,0,Conference,NaN,NaN,NaN


In [30]:
# getting information about dataframe

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4894402 entries, 0 to 4894401
Data columns (total 8 columns):
 #   Column           Dtype  
---  ------           -----  
 0   id               object 
 1   title            object 
 2   year             float64
 3   n_citation       object 
 4   doc_type         object 
 5   reference_count  object 
 6   references       object 
 7   doi              object 
dtypes: float64(1), object(7)
memory usage: 298.7+ MB


**Handling Null Values**

In [31]:
df.isnull().sum()

id                       0
title                   53
year                   330
n_citation             325
doc_type            499813
reference_count    1117295
references         1117264
doi                 973707
dtype: int64

In [32]:
df = df.dropna(subset=['title'])

****

In [33]:
#Creating some column to know the words  distribution

df["title_len"]=df["title"].apply(lambda x:len(x))

#Counting words
df["word_count"]=df["title"].apply(lambda x:(len(x.split())))


In [34]:
# Function for getting average word lenghth

def get_avg_word_len(x):
    
    words=x.split()
    word_len=0
    for word in words:
        word_len=word_len +len(word)
    return word_len/len(words) 

In [35]:
df["avg_word_len"]=df["title"].apply(lambda x: get_avg_word_len(x))

In [36]:
df.head()

,id,title,year,n_citation,doc_type,reference_count,references,doi,title_len,word_count,avg_word_len
0,1091,Preliminary Design of a Network Protocol Learn...,2013.0,1,Conference,2.0,2005687710;2018037215,https://doi.org/10.1007/978-3-642-39476-8_19,159,26,5.153846
1,1388,Further Results on Independence in Direct-Prod...,2000.0,1,Journal,NaN,NaN,NaN,57,7,7.285714
2,1674,A methodology for the physically accurate visu...,2011.0,1,Conference,15.0,1535888970;1992876689;1993710814;2035653341;20...,https://doi.org/10.2312/VAST/VAST11/137-144,84,11,6.727273
3,1688,"Comparison of GARCH, Neural Network and Suppor...",2009.0,6,Conference,3.0,1560724230;1986968751;2156909104,https://doi.org/10.1007/978-3-642-11164-8_97,98,14,6.071429
4,5411,COMPARING GNG3D AND QUADRIC ERROR METRICS METH...,2009.0,0,Conference,NaN,NaN,NaN,71,11,5.545455


**Some  Preprocessing steps**

In [37]:
 # converting title column to lower case

df["title"].str.lower()

0          preliminary design of a network protocol learn...
1          further results on independence in direct-prod...
2          a methodology for the physically accurate visu...
3          comparison of garch, neural network and suppor...
4          comparing gng3d and quadric error metrics meth...
                                 ...                        
4894397    a naïve-bayesian methodology to classify echo ...
4894398     blind equalization for underwater communications
4894399             technical implementation and feasibility
4894400                      knowledge patterns in databases
4894401    bemerkung zu gurevich's arbeit über das entsch...
Name: title, Length: 4894349, dtype: object

In [38]:
#Tag Removing Function (for safety purpose)

def remove_tags(text):
    pattern=re.compile("<.-*?>")
    return pattern.sub(r'',text)

In [39]:
#Tag Removal operation

df['title']=df['title'].apply(remove_tags)
df["title"]

0          Preliminary Design of a Network Protocol Learn...
1          Further Results on Independence in Direct-Prod...
2          A methodology for the physically accurate visu...
3          Comparison of GARCH, Neural Network and Suppor...
4          COMPARING GNG3D AND QUADRIC ERROR METRICS METH...
                                 ...                        
4894397    A Naïve-Bayesian methodology to classify echo ...
4894398     Blind Equalization for underwater communications
4894399             Technical Implementation and Feasibility
4894400                      Knowledge Patterns in Databases
4894401    Bemerkung zu Gurevich's Arbeit über das Entsch...
Name: title, Length: 4894349, dtype: object

**Removing Punctuation**

In [40]:
#Getting all the punctuation
punctuation=string.punctuation

In [41]:
# Implementing String Removing
def remove_punc(text): 
    return  text.translate(str.maketrans('','',punctuation))

In [42]:
df['title']=df['title'].apply(remove_punc)


In [43]:
df["title"]

0          Preliminary Design of a Network Protocol Learn...
1          Further Results on Independence in DirectProdu...
2          A methodology for the physically accurate visu...
3          Comparison of GARCH Neural Network and Support...
4          COMPARING GNG3D AND QUADRIC ERROR METRICS METH...
                                 ...                        
4894397    A NaïveBayesian methodology to classify echo c...
4894398     Blind Equalization for underwater communications
4894399             Technical Implementation and Feasibility
4894400                      Knowledge Patterns in Databases
4894401    Bemerkung zu Gurevichs Arbeit über das Entsche...
Name: title, Length: 4894349, dtype: object

In [44]:
#Example of Stemming
from nltk.stem import PorterStemmer
porter = PorterStemmer()
porter.stem('dancing')

# porter.stem('running')  o/p- run
#it used for finding root word

'danc'

In [45]:
import nltk
#nltk.download('omw-1.4')

In [46]:
#Examples of Lemmatization
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
print(lemmatizer.lemmatize("cats"))
print(lemmatizer.lemmatize("cacti"))

cat
cactus


## Building our recommendation model 

In [47]:
#Stemming

def stem(text):
    lst=[]
    for i in text.split():
        lst.append(ps.stem(i))
    return " ".join(lst)

In [48]:
df["title"]=df["title"].apply(lambda x:x.lower())

## Removing Stop Words

In [49]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=500,stop_words="english")

In [ ]:
# Converting titles into array
vectors=cv.fit_transform(df["title"]).toarray()

In [26]:
#Getting shape of vector
vectors.shape

(10000, 5000)

In [27]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [29]:
# Lấy tên các đặc trưng
feature_names = cv.get_feature_names_out()
print("Feature names:", feature_names)


Feature names: ['01' '10' '15' ... 'zu' 'zum' 'zur']


In [30]:
#Finding similarity between vectors

similarity=cosine_similarity(vectors)


In [31]:
# Acessing other similar element to "5th title"
similarity[5]

array([0., 0., 0., ..., 0., 0., 0.])

In [43]:
#Recommendation Function

def recommend(tag):
    
    title_index=df[df["title"]==tag].index[0]
    distances=similarity[title_index]
    title_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in title_list:
        print(df.iloc[i[0]].title)
    

In [49]:
def recommend(tag):
    if tag not in df['title'].values:
        print(f"Tiêu đề '{tag}' không tồn tại trong Data")
        return
    
    title_index = df[df["title"] == tag].index[0]
    distances = similarity[title_index]
    
    # Lấy 5 tiêu đề tương tự nhất
    title_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    print(f"Gợi ý cho '{tag}':")
    for i in title_list:
        print(f"- {df.iloc[i[0]].title} (Độ tương đồng: {i[1]:.4f})")  # In thêm độ tương đồng


In [44]:
df['title'][0]

'preliminary design of a network protocol learning tool based on the comprehension of high school students design by an empirical study using a simple mind map'

In [54]:
#top 5 similar type of title  
recommend('vectorial fast correlation attacks')

Gợi ý cho 'vectorial fast correlation attacks':
- fast dqbf refutation (Độ tương đồng: 0.5774)
- boomerang attacks on blake32 (Độ tương đồng: 0.5774)
- suicide bombing attacks simulation (Độ tương đồng: 0.4082)
- fast piercing of isooriented rectangles (Độ tương đồng: 0.4082)
- fast wireframevisibility algorithm (Độ tương đồng: 0.4082)


****

In [55]:
#internal things happening inside above function

x=df[df["title"]=='vectorial fast correlation attacks'].index[0]
distances=similarity[x]
title_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:15]
title_list

for i in title_list:
    print([i[0]])

[6700]
[7582]
[3851]
[6774]
[7170]
[975]
[1230]
[1776]
[2762]
[2796]
[5576]
[5585]
[8932]
[9714]
